In [ ]:
%matplotlib inline

In [ ]:
import gc
import pathlib
import sys

In [ ]:
import joblib
import lightgbm as lgb
import pandas as pd

In [ ]:
root_dir = "../.."

In [ ]:
root_dir_path = pathlib.Path(root_dir)
data_dir_path = root_dir_path / "data"
processed_dir_path = data_dir_path / "processed"
train_path = processed_dir_path / "train.parquet"
models_dir_path = root_dir_path / "models"
model_path = models_dir_path / "model.joblib"
src_dir_path = root_dir_path / "src"

In [ ]:
sys.path.append(str(src_dir_path))

In [ ]:
from package.constants import *

In [ ]:
train = pd.read_parquet(train_path)

In [ ]:
# See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/135896
is_train = (train["date"] >= "2015-03-28") & (train["date"] < "2016-03-28")
is_valid = train["date"] >= "2016-03-28"

In [ ]:
dtrain = lgb.Dataset(
    train.loc[is_train, features],
    train.loc[is_train, target],
    weight=train.loc[is_train, "sample_weight"],
    categorical_feature=categorical_features,
)
dvalid = lgb.Dataset(
    train.loc[is_valid, features],
    train.loc[is_valid, target],
    weight=train.loc[is_valid, "sample_weight"],
    categorical_feature=categorical_features,
    reference=dtrain,
)

In [ ]:
%%time
params = {
    "feature_fraction": 0.5,
    "learning_rate": 0.05,
    "metric": "rmse",
    "n_jobs": -1,
    "objective": "regression",
    "seed": 0,
}
evals_result = {}
model = lgb.train(
    params,
    dtrain,
    valid_sets=[dvalid],
    num_boost_round=10_000,
    early_stopping_rounds=100,
    evals_result=evals_result,
)

In [ ]:
ax = lgb.plot_metric(evals_result, metric="rmse", figsize=(16, 9))

In [ ]:
ax = lgb.plot_importance(model, importance_type="split", figsize=(16, 9))

In [ ]:
ax = lgb.plot_importance(model, importance_type="gain", figsize=(16, 9))

In [ ]:
is_train |= is_valid

In [ ]:
dtrain = lgb.Dataset(
    train.loc[is_train, features],
    train.loc[is_train, target],
    weight=train.loc[is_train, "sample_weight"],
    categorical_feature=categorical_features,
)

In [ ]:
%%time
model = lgb.train(params, dtrain, num_boost_round=model.best_iteration)

In [ ]:
joblib.dump(model, model_path)